In [177]:
t=30
n=10
eeg = np.arange(0,t)
speech = np.arange(0,t)

B=[]
for i in range(n):
    speech = np.roll(speech,1)
    A = np.vstack((eeg,speech))
    B.append(A)
    

In [178]:
B

[array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [29,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
         15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]),
 array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [28, 29,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
         14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]]),
 array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [27, 28, 29,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
         13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]]),
 array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [26, 27, 28, 

In [277]:
t=124
n=100
b = np.arange(0,t)
a = np.arange(0,t)

A=[]
k=1
for a in multiset_permutations(a):
    c = b-a
    if not np.any(c == 0):
        if not a in A:
            A.append(a)
            print(0,end =' ')


KeyboardInterrupt: 

In [260]:
t=124
n=100

b = np.arange(0,t)
a = np.arange(0,t)

A=[]
for i in range(n):
    while True:
        np.random.shuffle(a)
        c = b-a
        
        if not np.any(c == 0):
            if not a in A:
                A.append(a)
                A = np.unique(A,axis=0)
                break

2 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

 2132 2133 2134 2135 2136 2137 2138 2139 2140 2141 2142 2143 2144 2145 2146 2147 2148 2149 2150 2151 2152 2153 2154 2155 2156 2157 2158 2159 2160 2161 2162 2163 2164 2165 2166 2167 2168 2169 2170 2171 2172 2173 2174 2175 2176 2177 2178 2179 2180 2181 2182 2183 2184 2185 2186 2187 2188 2189 2190 2191 2192 2193 2194 2195 2196 2197 2198 2199 2200 2201 2202 2203 2204 2205 2206 2207 2208 2209 2210 2211 2212 2213 2214 2215 2216 2217 2218 2219 2220 2221 2222 2223 2224 2225 2226 2227 2228 2229 2230 2231 2232 2233 2234 2235 2236 2237 2238 2239 2240 2241 2242 2243 2244 2245 2246 2247 2248 2249 2250 2251 2252 2253 2254 2255 2256 2257 2258 2259 2260 2261 2262 2263 2264 2265 2266 2267 2268 2269 2270 2271 2272 2273 2274 2275 2276 2277 2278 2279 2280 2281 2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294 2295 2296 2297 2298 2299 2300 2301 2302 2303 2304 2305 2306 2307 2308 2309 2609 2610 2611 2612 2613 2614 2615 2616 2617 2618 2619 2620 2621 2622 2623 2624 2625 2626 2627 2628 2629 2630

 5460 5461 5462 5463 5464 5465 5466 5467 5468 5469 5470 5471 5472 5473 5474 5475 5476 5477 5478 5479 5480 5481 5482 5483 5484 5485 5486 5487 5488 5489 5490 5491 5492 5493 5494 5495 5496 5497 5498 5811 5812 5813 5814 5815 5816 5817 5818 5819 5820 5821 5822 5823 5824 5825 5826 5827 5828 5829 5830 5831 5832 5833 5834 5835 5836 5837 5838 5839 5840 5841 5842 5843 5844 5845 5846 5847 5848 5849 5850 5851 5852 5853 5854 5855 5856 5857 5858 5859 5860 5861 5862 5863 5864 5865 5866 5867 5868 5869 5870 5871 5872 5873 5874 5875 5876 5877 5878 5879 5880 5881 5882 5883 5884 5885 5886 5887 5888 5889 5890 5891 5892 5893 5894 5895 5896 5897 5898 5899 5900 5901 5902 5903 5904 5905 5906 5907 5908 5909 5910 5911 5912 5913 5914 5915 5916 5917 5918 5919 5920 5921 5922 5923 5924 5925 5926 5927 5928 5929 5930 5931 5932 5933 5934 5935 5936 5937 5938 5939 5940 5941 5942 5943 5944 5945 5946 5947 5948 5949 5950 5951 5952 5953 5954 5955 5956 5957 5958 5959 5960 5961 5962 5963 5964 5965 5966 5967 5968 5969 5970 5971

 8959 8960 8961 8962 8963 8964 8965 8966 8967 8968 8969 8970 8971 8972 8973 8974 8975 8976 8977 8978 8979 8980 8981 8982 8983 8984 8985 8986 8987 8988 8989 8990 8991 8992 8993 8994 8995 8996 8997 8998 8999 9000 9001 9002 9003 9004 9005 9006 9007 9008 9009 9010 9011 9012 9013 9014 9015 9016 9017 9018 9019 9020 9021 9022 9023 9024 9025 9026 9027 9028 9029 9030 9031 9032 9033 9034 9035 9036 9037 9038 9039 9040 9041 9042 9043 9044 9045 9046 9047 9048 9049 9050 9051 9052 9053 9054 9055 9056 9057 9058 9059 9060 9061 9062 9063 9064 9065 9066 9067 9068 9069 9070 9071 9072 9073 9074 9075 9076 9077 9078 9079 9080 9081 9082 9083 9084 9085 9086 9087 9088 9089 9090 9091 9092 9093 9094 9095 9096 9097 9098 9099 9100 9101 9102 9103 9104 9105 9106 9107 9108 9345 9346 9347 9348 9349 9350 9351 9352 9353 9354 9355 9356 9357 9358 9359 9360 9361 9362 9363 9364 9365 9366 9367 9368 9369 9370 9371 9372 9373 9374 9375 9376 9377 9378 9379 9380 9381 9382 9383 9384 9385 9386 9387 9388 9389 9390 9391 9392 9393 9394

 11915 11916 11917 11918 11919 11920 11921 11922 11923 11924 11925 11926 11927 11928 11929 11930 11931 11932 11933 11934 11935 11936 11937 11938 11939 11940 11941 11942 11943 11944 11945 11946 11947 11948 11949 11950 11951 11952 11953 11954 11955 11956 11957 11958 11959 11960 11961 11962 11963 11964 11965 11966 11967 11968 11969 11970 11971 11972 11973 11974 11975 11976 11977 11978 11979 11980 11981 11982 11983 11984 11985 11986 11987 11988 11989 11990 11991 11992 11993 11994 11995 11996 11997 11998 11999 12000 12001 12002 12003 12004 12005 12006 12007 12008 12009 12010 12011 12012 12013 12014 12015 12016 12017 12018 12019 12020 12021 12022 12023 12024 12025 12026 12027 12028 12029 12030 12031 12032 12033 12034 12035 12036 12037 12038 12039 12040 12041 12042 12043 12044 12045 12046 12047 12048 12049 12050 12051 12052 12053 12054 12055 12056 12057 12058 12059 12060 12061 12062 12063 12064 12065 12066 12067 12068 12069 12070 12071 12072 12073 12074 12075 12076 12077 12078 12079 12080 120

...

 37834 37835 37836 37837 37838 37839 37840 37841 37842 37843 37844 37845 37846 37847 37848 37849 37850 37851 37852 37853 37854 37855 37856 37857 37858 37859 37860 37861 37862 37863 37864 37865 37866 37867 37868 37869 37870 37871 37872 37873 37874 37875 37876 37877 37878 37879 37880 37881 37882 37883 37884 37885 37886 37887 37888 37889 37890 37891 37892 37893 37894 37895 37896 37897 37898 37899 37900 37901 37902 37903 37904 37905 37906 37907 37908 37909 37910 37911 37912 37913 37914 37915 37916 37917 37918 37919 37920 37921 37922 37923 37924 37925 37926 37927 37928 37929 37930 37931 37932 37933 37934 37935 37936 37937 37938 37939 37940 37941 37942 37943 37944 37945 37946 37947 37948 37949 37950 37951 37952 37953 37954 37955 37956 37957 37958 37959 37960 37961 37962 37963 37964 37965 37966 37967 37968 37969 37970 37971 37972 37973 37974 37975 37976 37977 37978 37979 37980 37981 37982 37983 38247 38248 38249 38250 38251 38252 38253 38254 38255 38256 38257 38258 38259 38260 38261 38262 382

 40575 40576 40577 40578 40579 40580 40581 40582 40583 40584 40585 40586 40587 40588 40589 40590 40591 40592 40593 40594 40595 40596 40597 40598 40599 40600 40601 40602 40603 40604 40605 40606 40607 40608 40891 40892 40893 40894 40895 40896 40897 40898 40899 40900 40901 40902 40903 40904 40905 40906 40907 40908 40909 40910 40911 40912 40913 40914 40915 40916 40917 40918 40919 40920 40921 40922 40923 40924 40925 40926 40927 40928 40929 40930 40931 40932 40933 40934 40935 40936 40937 40938 40939 40940 40941 40942 40943 40944 40945 40946 40947 40948 40949 40950 40951 40952 40953 40954 40955 40956 40957 40958 40959 40960 40961 40962 40963 40964 40965 40966 40967 40968 40969 40970 40971 40972 40973 40974 40975 40976 40977 40978 40979 40980 40981 40982 40983 40984 40985 40986 40987 40988 40989 40990 40991 40992 40993 40994 40995 40996 40997 40998 40999 41000 41001 41002 41003 41004 41005 41006 41007 41008 41009 41010 41011 41012 41013 41014 41015 41016 41017 41018 41019 41020 41021 41022 410

 43691 43692 43693 43694 43695 43696 43697 43698 43699 43700 43701 43702 43703 43704 43705 43706 43707 43708 43709 43710 43711 43712 43713 43714 43715 43716 43717 43718 43719 43720 43721 43722 43723 43724 43725 43726 43727 43728 43729 43730 43731 43732 43733 43734 43735 43736 43737 43738 43739 43740 43741 43742 43743 43744 43745 43746 43747 43748 43749 43750 43751 43752 43753 43754 43755 43756 43757 43758 43759 43760 43761 43762 43763 43764 43765 43766 43767 43768 43769 43770 43771 43772 43773 43774 43775 43776 43777 43778 43779 43780 43781 43782 43783 43784 43785 43786 43787 43788 43789 43790 43791 43792 43793 43794 43795 43796 43797 43798 43799 43800 43801 43802 43803 43804 43805 43806 43807 43808 43809 43810 43811 43812 43813 43814 43815 43816 43817 43818 43819 43820 43821 43822 43823 43824 43825 43826 43827 43828 43829 43830 43831 43832 43833 43834 43835 43836 43837 43838 43839 43840 43841 43842 43843 43844 43845 43846 43847 43848 43849 43850 43851 43852 43853 43854 43855 43856 438

 46546 46547 46548 46549 46550 46551 46552 46553 46554 46555 46556 46557 46558 46559 46560 46561 46562 46563 46564 46565 46566 46567 46568 46569 46570 46571 46572 46573 46574 46575 46576 46577 46578 46579 46580 46581 46582 46583 46584 46585 46586 46587 46588 46589 46590 46591 46592 46593 46594 46595 46596 46597 46859 46860 46861 46862 46863 46864 46865 46866 46867 46868 46869 46870 46871 46872 46873 46874 46875 46876 46877 46878 46879 46880 46881 46882 46883 46884 46885 46886 46887 46888 46889 46890 46891 46892 46893 46894 46895 46896 46897 46898 46899 46900 46901 46902 46903 46904 46905 46906 46907 46908 46909 46910 46911 46912 46913 46914 46915 46916 46917 46918 46919 46920 46921 46922 46923 46924 46925 46926 46927 46928 46929 46930 46931 46932 46933 46934 46935 46936 46937 46938 46939 46940 46941 46942 46943 46944 46945 46946 46947 46948 46949 46950 46951 46952 46953 46954 46955 46956 46957 46958 46959 46960 46961 46962 46963 46964 46965 46966 46967 46968 46969 46970 46971 46972 469

 49550 49551 49552 49553 49554 49555 49556 49557 49558 49559 49560 49561 49562 49563 49564 49565 49566 49567 49568 49569 49570 49571 49572 49573 49574 49575 49576 49577 49578 49579 49580 49581 49582 49583 49584 49585 49586 49587 49588 49589 49590 49591 49592 49593 49594 49595 49596 49597 49598 49599 49600 49601 49602 49603 49604 49605 49606 49607 49608 49609 49610 49611 49612 49613 49614 49615 49616 49617 49618 49619 49620 49621 49622 49623 49624 49625 49626 49627 49628 49629 49630 49631 49632 49633 49634 49635 49636 49637 49638 49639 49640 49641 49642 49643 49644 49645 49646 49647 49648 49649 49650 49651 49652 49653 49654 49655 49656 49657 49658 49659 49660 49661 49662 49663 49664 49665 49666 49667 49668 49669 49670 49671 49672 49673 49674 49675 49676 49677 49678 49679 49680 49681 49682 49683 49684 49685 49686 49687 49688 49689 49690 49691 49692 49693 49694 49695 49696 49697 49698 49699 49700 49701 49702 49703 49704 49705 49706 49707 49708 49709 49710 49711 49712 49713 49714 49715 497

In [173]:
t=10

from itertools import permutations
c= list(permutations(np.arange(0,t), 2))

a = np.asarray(c)
a.shape

(90, 2)

In [174]:
B=np.zeros((10,t,2))
X = np.arange(0,t)
for j in range(10):
    A=[]
    EEG = []
    Speech = []
    while True:
        X = np.roll(X,1)
        for i in range(0,len(X)):
                b = np.where(a[:,0]==X[i])
                if not len(b[0]) == 0:
                    for k in range(len(b[0])):
                        if not a[b[0][k],1] in Speech:
                            A.append(a[b[0][k],:])
                            EEG.append(a[b[0][k],0])
                            Speech.append(a[b[0][k],1])
                            a = np.delete(a, b[0][k], 0)
                            break

    A = np.asarray(A)
    if A.shape[0] == t:
        B[j,:,:]=A
    else:
        print('..........')

..........


In [170]:
B

array([[[9., 0.],
        [0., 1.],
        [1., 2.],
        [2., 3.],
        [3., 4.],
        [4., 5.],
        [5., 6.],
        [6., 7.],
        [7., 8.],
        [8., 9.]],

       [[8., 0.],
        [9., 1.],
        [0., 2.],
        [1., 3.],
        [2., 4.],
        [3., 5.],
        [4., 6.],
        [5., 7.],
        [6., 8.],
        [7., 9.]],

       [[7., 0.],
        [8., 1.],
        [9., 2.],
        [0., 3.],
        [1., 4.],
        [2., 5.],
        [3., 6.],
        [4., 7.],
        [5., 8.],
        [6., 9.]],

       [[6., 0.],
        [7., 1.],
        [8., 2.],
        [9., 3.],
        [0., 4.],
        [1., 5.],
        [2., 6.],
        [3., 7.],
        [4., 8.],
        [5., 9.]],

       [[5., 0.],
        [6., 1.],
        [7., 2.],
        [8., 3.],
        [9., 4.],
        [0., 5.],
        [1., 6.],
        [2., 7.],
        [3., 8.],
        [4., 9.]],

       [[4., 0.],
        [5., 1.],
        [6., 2.],
        [7., 3.],
        [8., 4.],


In [151]:
ii = np.arange(0,len(X)-1)
i1=ii[0]
i2=ii[-1]
ii[-1]=i1
ii[0]=i2
X[ii]
X


array([0, 1, 2, 3, 4])

In [159]:
X = np.arange(0,5)
X

array([0, 1, 2, 3, 4])

In [167]:
X = np.roll(X,1)
X

array([0, 1, 2, 3, 4])

In [123]:
B=np.zeros((10,t,2))
for j in range(10):
    A=[]
    EEG = []
    Speech = []
    for i in range(t):
        while True:
            i = np.random.randint(0,t, size=1)
            b = a[i[0]]
            if not b[0] in EEG and b[1] in Speech:
                A.append(b)
                EEG.append(b[0])
                Speech.append(b[1])
                a = np.delete(a, i, 0)
                break
    B[j,:,:]=A

KeyboardInterrupt: 

In [125]:
A

[]

In [94]:
B

array([[[0., 1.],
        [1., 0.],
        [2., 3.],
        [3., 2.],
        [4., 5.],
        [5., 4.],
        [6., 7.],
        [7., 6.],
        [8., 9.],
        [9., 8.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],


In [91]:
A.shape[0]

8

In [87]:
B

array([[[0., 1.],
        [1., 0.],
        [2., 3.],
        [3., 2.],
        [4., 5.],
        [5., 4.],
        [6., 7.],
        [7., 6.],
        [8., 9.],
        [9., 8.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],


In [264]:
from sympy.utilities.iterables import multiset_permutations

In [5]:
B

array([[[  0.,   1.],
        [  1.,   0.],
        [  2.,   0.],
        ...,
        [117.,   0.],
        [118.,   0.],
        [119.,   0.]],

       [[  0.,   2.],
        [  1.,   2.],
        [  2.,   1.],
        ...,
        [117.,   1.],
        [118.,   1.],
        [119.,   1.]],

       [[  0.,   3.],
        [  1.,   3.],
        [  2.,   3.],
        ...,
        [117.,   2.],
        [118.,   2.],
        [119.,   2.]],

       ...,

       [[  0.,   8.],
        [  1.,   8.],
        [  2.,   8.],
        ...,
        [117.,   7.],
        [118.,   7.],
        [119.,   7.]],

       [[  0.,   9.],
        [  1.,   9.],
        [  2.,   9.],
        ...,
        [117.,   8.],
        [118.,   8.],
        [119.,   8.]],

       [[  0.,  10.],
        [  1.,  10.],
        [  2.,  10.],
        ...,
        [117.,   9.],
        [118.,   9.],
        [119.,   9.]]])

In [183]:
A=[]
for i in range(t):
    b = np.where(a[:,0]==i)
    if not len(b[0]) == 0:
        A.append(a[b[0][0],:])
        a = np.delete(a, b[0][0], 0)
A = np.asarray(A)

In [60]:
i=0
b = np.where(a[:,0]==i)
a = np.delete(a, b[0][0], 0)
a.shape

(7625, 2)

In [234]:
A[0]

array([ 70,  48,  88,  74,  40,  35,  47,  92,  90,   4,  54,  64,  91,
       106,  46,  45,  19,  20,  28,  49,   6,  71, 121,  96, 117,  83,
        12,  89, 101,   8, 105,   9,  38,  41,   3,   2,  99,  10, 114,
       104,  78, 116,  63,  97,   1,  24,  33,  59, 107,  53,  51,  26,
        17,  50,  77, 113,  95, 119,  21, 118, 102, 110,  43,  75,  84,
       100,  36,  93,  80, 103,  61,  52,  23, 109,  87,  60,  11,  65,
       115,  81,   0,  22, 120,  31,  32,  37,  85,  68,  76,  82,  25,
        15,  67,  55, 112, 108,  30,  72,  66,  34,  14,  62,  58,  56,
        27,  73,  44,  42,  13,  98,  86,  39, 123,   7,  69,  29,   5,
        16, 122,  94,  57,  18, 111,  79])

In [23]:
a = np.arange(0,3)
b = np.arange(0,3)
c = [list(zip(a, p)) for p in permutations(b)]


6

In [17]:
a = np.ndarray(shape=(124,2),dtype=int)
a[:,0] = np.arange(0,124)
a[:,1] = np.arange(0,124)
a.shape

(124, 2)

In [2]:
import mne
import os
import scipy.io
import listen_italian_functions
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')
from tqdm import tnrange, tqdm_notebook

data_path = os.path.dirname(os.path.dirname(os.getcwd()))

subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale','Sara',
                'Silvia','Silvia2','Tommaso']
#subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
#                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale',
#                'Silvia','Silvia2','Tommaso']


# epoching and saving

In [ ]:
# extract trials of tmax second and remove the wrong answer trials and seperate them in three conditions
Tmin = 0
Tmax = 3.51
trial_len = 2

for s in subject_name:
    raw_fname = data_path + '/python/data/'+s+'_raw.fif'
    raw = mne.io.read_raw_fif(raw_fname,preload=True)
    raw_fname = data_path + '/behaviour/data/subject/'+s+'_behaviour.mat'
    mat = scipy.io.loadmat(raw_fname)
    epochs = listen_italian_functions.epoch(raw, mat,Tmin, Tmax)
    save_path = data_path + '/python/data/coherence_epochs/'+s+'-coh-epo-'+str(Tmin)+'-'+str(Tmax)+'-trialLen-'+str(trial_len)+'.fif'
    epochs.save(save_path)
    print('----------------------------------------------------------------------------------------------------------------'+s)



# Read the epoches

In [236]:
Tmin = 0
Tmax = 3.51
trial_len = 2

GA_epoches = []
for s in subject_name:
    save_path = data_path + '/python/data/coherence_epochs/'+s+'-coh-epo-'+str(Tmin)+'-' \
    +str(Tmax)+'-trialLen-'+str(trial_len)+'.fif'
    epochs = mne.read_epochs(save_path)
    GA_epoches.append(epochs)
    print('----------------------------------------------------------------------------------------------------------------'+s)


This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Alice-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Alice-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
124 matching events found
No baseline correction applied
124 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Alice
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Andrea-coh-epo-0-3.51-trialLen-2.fif) doe

Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pagani-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
122 matching events found
No baseline correction applied
122 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Pagani
This filename (C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pasquale-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading C:\Users\SMukherjee\Desktop\projects\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pasquale-coh-epo-0-3.51-trialLen-2.fif ...


In [ ]:
condition = ['Hyper','Normal','Hypo']
frames = []
for s in range(0,len(subject_name)):
    df = pd.DataFrame({'Condition':'Hyper','Subject':subject_name[s],'noTrials':GA_epoches[s]['hyper'].get_data().shape[0]},index=[s])
    df = df.append(pd.DataFrame({'Condition':'Normal','Subject':subject_name[s],'noTrials':GA_epoches[s]['normal'].get_data().shape[0]},index=[s]))
    df = df.append(pd.DataFrame({'Condition':'Hypo','Subject':subject_name[s],'noTrials':GA_epoches[s]['hypo'].get_data().shape[0]},index=[s]))
    frames.append(df)
    
data=pd.concat((frames),axis=0)

In [ ]:
a=data.groupby(['Subject','Condition']).sum()['noTrials'].unstack().plot(kind='bar',figsize=(20,5),grid=True)

# Measuring the coherence

In [ ]:
remove_first = 0.5 #seconds

# let's explore some frequency bands
iter_freqs = [
    ('Delta', 1, 3),
    ('Theta', 4, 7),
    ('Alpha', 8, 12),
    ('Beta', 13, 25),
    ('Gamma', 30, 40)
]

iter_freqs = [
    ('fr', 1, 3),
    #('fr', 2, 4),
    #('fr', 3, 5),
    ('fr', 4, 6)
    #('fr', 5, 7),
    #('fr', 6, 8),
    #('fr', 7, 9),
    #('fr', 8, 10),
    #('fr', 9, 11),
    #('fr', 10, 12)
]
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
condition = ['Hyper','Normal','Hypo', 'All']
condition_dummy = ['hyper','normal','hypo']

delay = np.arange(0,1.1,0.1)


indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))   

extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'Hyper': 1,'Normal': 2,'Hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
        
for s in range(0,len(subject_name)):
    frames = []
    for d in delay:        

        Ocoh = listen_italian_functions.coherence_preprocess_delay(GA_epoches[s],remove_first,d,trial_len,
                                                                        extra_channels,eeg_channles,
                                                                        info,ch_names,event_id,iter_freqs,indices,condition)

            
            
        for i in range(0,len(condition)):
            # mean or median of the surrogate distribution
            coh=Ocoh[:,:,i]
            for fr in range(0,len(iter_freqs)):
                a = str(iter_freqs[fr][0])+ ' '+str(iter_freqs[fr][1])+' - '+str(iter_freqs[fr][2])+'Hz'
                cc = np.split(coh[:,fr], len(features))
                for f in range(0,len(features)):
                    feature = features[f]
                    if condition[i] != 'All':
                        df = pd.DataFrame({'Condition':condition[i],'Freq':a,'Delay':d,'Subject':subject_name[s],'Feature':feature,
                                       'Data':[cc[f].flatten()],'noTrials':GA_epoches[s][condition_dummy[i]].get_data().shape[0],})
                    else:
                        df = pd.DataFrame({'Condition':condition[i],'Freq':a,'Delay':d,'Subject':subject_name[s],'Feature':feature,
                                       'Data':[cc[f].flatten()],'noTrials':GA_epoches[s].get_data().shape[0],})

            
                    
                    frames.append(df) 
    print(d)
                
    data=pd.concat((frames),axis=0)
    save_path = data_path + '/python/data/1-12Hz_frequency_bands_in2hzStep_coherence/coherence-trialLen-'\
    +str(trial_len)+'-removedFirst-'+str(remove_first)+'s-'+subject_name[s]
    data.to_pickle(save_path)

# Measure the coherence in specific delay and shifting the eeg and speech from speech onset

In [ ]:
remove_first = [0,0.1,0.2,0.3,0.4,0.5] #seconds

for rf in remove_first:
        # let's explore some frequency bands
    iter_freqs = [
        ('Delta', 1, 3),
        ('Theta', 4, 7),
        ('Alpha', 8, 12),
        ('Beta', 13, 25),
        ('Gamma', 30, 40)
    ]
    features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
    condition = ['Hyper','Normal','Hypo']
    delay = np.arange(0,1.1,0.1)
    delay = [0,0.1,0.2]
    delay = np.add(delay,0.5) #shift to below algorithm format
    indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))  
    extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
    eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
    event_id = {'hyper': 1,'normal': 2,'hypo': 3}
    ch_types = np.repeat('eeg', len(features)+59)
    ch_names = np.hstack((eeg_channles,features))        
    info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
    ch_names = np.setdiff1d(extra_channels,features)

    for s in tqdm_notebook(range(0,len(subject_name)), desc='Subjects'):
        frames = []
        for d in tqdm_notebook(delay, desc='Delay'):        

            epoch = listen_italian_functions.coherence_preprocess_delay(GA_epoches[s],rf,d,trial_len,
                                                                        extra_channels,eeg_channles,info,ch_names,event_id)


            for band, fmin, fmax in iter_freqs:
                a = band+ ' '+str(fmin)+' - '+str(fmax)+'Hz'                
                hyper, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['hyper'],fmin, fmax,indices)
                normal, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['normal'],fmin, fmax,indices)
                hypo, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch['hypo'],fmin, fmax,indices)
                allC, freqs, times, n_epochs, n_tapers = listen_italian_functions.coherence_measure(epoch,fmin, fmax,indices)

                hyper = np.split(hyper, len(features))
                normal = np.split(normal, len(features))
                hypo = np.split(hypo, len(features))
                allC = np.split(allC, len(features))

                for f in range(0,len(features)):
                    feature = features[f]
                    df = pd.DataFrame({'Condition':'Hyper','Freq':a,'Delay':d,'Feature':feature,
                                               'noTrials':GA_epoches[s]['hyper'].get_data().shape[0],
                                               'Subject':subject_name[s],'Data':[hyper[f].flatten()]})
                    df = df.append(pd.DataFrame({'Condition':'Normal','Freq':a,'Delay':d,'Feature':feature,
                                                 'noTrials':GA_epoches[s]['normal'].get_data().shape[0],
                                                 'Subject':subject_name[s],'Data':[normal[f].flatten()]}))
                    df = df.append(pd.DataFrame({'Condition':'Hypo','Freq':a,'Delay':d,'Feature':feature,
                                                 'noTrials':GA_epoches[s]['hypo'].get_data().shape[0],
                                                 'Subject':subject_name[s],'Data':[hypo[f].flatten()]}))
                    df = df.append(pd.DataFrame({'Condition':'All','Freq':a,'Delay':d,'Feature':feature,
                                                 'noTrials':GA_epoches[s].get_data().shape[0],
                                                 'Subject':subject_name[s],'Data':[allC[f].flatten()]}))
                    frames.append(df)
            print(d)

        data=pd.concat((frames),axis=0)
        save_path = data_path + '/analysis/python/data/shift/coherence-trialLen-'+str(trial_len)+'-removedFirst-'+str(rf)+'s-'+subject_name[s]
        data.to_pickle(save_path)

# Bootstrapping and creating a surrogate distribution for each subject. substract the average of the distribution from the original coherence value.

In [ ]:
no_surrogates = 100
remove_first = 0.5 #seconds
iter_freqs = [
    ('fr', 1, 3),
    ('fr', 4, 6)
]
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
delay = np.arange(0,1.1,0.1)



indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))  
extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'hyper': 1,'normal': 2,'hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
    
for s in range(0,len(subject_name)):
    frames = []
    for d in delay:       
        surrogate_coh = listen_italian_functions.coherence_preprocess_delay_surrogate(GA_epoches[s],remove_first,
                                                                                      d,trial_len,extra_channels,
                                                                                      eeg_channles,info,ch_names,
                                                                                      event_id,no_surrogates,
                                                                                      iter_freqs,indices)
        
        # mean or median of the surrogate distribution
        coh=surrogate_coh.mean(axis=2)
        
        for fr in range(0,len(iter_freqs)):
            a = str(iter_freqs[fr][0])+ ' '+str(iter_freqs[fr][1])+' - '+str(iter_freqs[fr][2])+'Hz'
            cc = np.split(coh[:,fr], len(features))
            for f in range(0,len(features)):
                feature = features[f]
                df = pd.DataFrame({'Condition':'All','Freq':a,'Delay':d,'Subject':subject_name[s],'Feature':feature,
                                   'Surrogate':[cc[f].flatten()]})
                frames.append(df)                
    data=pd.concat((frames),axis=0)
    save_path = data_path + '/python/data/SurrogateCoherence/SurrogateCoherence-trialLen-'+str(trial_len)+'-removedFirst-'\
    +str(remove_first)+'s-'+subject_name[s]
    data.to_pickle(save_path)     
        
    

In [ ]:
delay = np.arange(0.5,1,0.1)
delay

In [ ]:
# putit into one file
save_path = data_path + '/python/data/SurrogateCoherence/SurrogateCoherence-trialLen-'+str(trial_len)+'-removedFirst-'\
    +str(remove_first)+'s'

frames = []
for s in subject_name:    
    frames.append(pd.read_pickle(save_path+'-'+s))
    
data = pd.concat((frames),axis=0)